# G4 - Generador de Jupyter Notebooks

Genera un Jupyter Notebook `.ipynb` a partir de un archivo de código Python 
`.py`, interpretando los comentarios como celdas Markdown y el código como 
celdas de código.

## Importar Librerías

`nbformat` es la librería estándar de Jupyter para **crear, leer y manipular** archivos `.ipynb`. Proporciona funciones para construir notebooks completos con celdas de código y markdown, siguiendo la especificación del formato notebook (v4).

In [1]:
import nbformat 

Rutas de entrada y salida

Se definen las rutas de los archivos:
- **`py_path`**: Ruta al archivo `.py` de entrada que se va a convertir.
- **`ipynb_path`**: Ruta donde se guardará el notebook generado.

In [2]:
py_path = "input/code-G4.py" 
ipynb_path = "output/G4-notebook.ipynb" 

## Lectura del archivo fuente

Se abre el archivo `.py` en modo lectura con codificación UTF-8 y se leen **todas las líneas** en una lista. Cada elemento de `lines` es una cadena que incluye el carácter de nueva línea (`\n`) al final.

In [3]:
with open(py_path, "r", encoding="utf-8") as f: 
    lines = f.readlines() 
 
print(f"Archivo leído: {py_path} ({len(lines)} líneas)") 

Archivo leído: input/code-G4.py (24 líneas)


## Creación del objeto Notebook

Se crea un **nuevo notebook vacío** usando `nbformat.v4.new_notebook()`.

In [4]:
nb = nbformat.v4.new_notebook() 

## Inicialización de variables de estado

Se declaran dos variables que mantienen el **estado** del procesamiento:

- **`buffer`**: Una lista que acumula las líneas pertenecientes a la celda que se está construyendo actualmente. Cada vez que se detecta un cambio de tipo o una línea vacía, el contenido del buffer se convierte en una celda y el buffer se vacía.
- **`current_type`**: Indica el tipo de la celda que se está construyendo (`"code"` o `"markdown"`). Se inicializa en `None` porque aún no se ha leído ninguna línea.

In [5]:
buffer = [] 
current_type = None  # "code" o "markdown" 

## Procesamemiento línea por línea


Si la línea está vacía (`line.strip() == ""`), se interpreta como un **separador entre celdas**:
- Si hay contenido acumulado en el `buffer`, se crea una celda del tipo correspondiente (`markdown` o `code`) y se agrega al notebook.
- El `buffer` se reinicia y `current_type` vuelve a `None`.
- Se usa `continue` para saltar al siguiente ciclo del bucle.


Para las líneas no vacías, se determina si es un **comentario** o **código**:
- Es **comentario** si empieza con `#` (ignorando espacios al inicio), pero **no** si empieza con `#!` (shebang, usado en scripts ejecutables de Unix).
- Los comentarios se asignan como tipo `"markdown"`, el resto como `"code"`.


Si `current_type` no es `None` y el tipo de la línea actual difiere del tipo anterior, significa que hubo una **transición** (por ejemplo, de comentario a código o viceversa). En ese caso:
- El contenido acumulado en el `buffer` se convierte en celda y se agrega al notebook.
- El `buffer` se reinicia para comenzar a acumular la nueva celda.


- **Si es markdown**: Se elimina el carácter `#` inicial y el espacio opcional que le sigue, dejando solo el texto del comentario. Esto permite que los comentarios del `.py` se muestren como texto limpio en las celdas Markdown.
- **Si es código**: La línea se agrega tal cual al buffer, preservando su formato original.

In [6]:
for line in lines:
    # Línea vacía: termina la celda actual
    if line.strip() == "":
        if buffer:
            contenido = "".join(buffer)
            if current_type == "markdown":
                nb.cells.append(nbformat.v4.new_markdown_cell(contenido))
            else:
                nb.cells.append(nbformat.v4.new_code_cell(contenido))
            buffer = []
            current_type = None
        continue

    # Determinar si es comentario o código
    is_comment = line.lstrip().startswith("#") and not line.lstrip().startswith("#!")
    line_type = "markdown" if is_comment else "code"

 
    # Si cambia el tipo, guardar celda anterior 
    if current_type is not None and line_type != current_type: 
        contenido = "".join(buffer) 
        if current_type == "markdown": 
            nb.cells.append(nbformat.v4.new_markdown_cell(contenido)) 
        else: 
            nb.cells.append(nbformat.v4.new_code_cell(contenido)) 
        buffer = [] 
 
    current_type = line_type 
 
    # Procesar la línea según su tipo 
    if line_type == "markdown": 
        text = line.lstrip()[1:]  # Quitar el '#' 
        if text.startswith(" "): 
            text = text[1:]       # Quitar el espacio después del '#' 
        buffer.append(text) 
    else: 
        buffer.append(line) 

## Vaciado del buffer residual

Después de terminar el bucle, es posible que el `buffer` aún contenga líneas de la **última celda** del archivo (si el archivo no termina con una línea vacía). Este bloque se asegura de que ese contenido residual no se pierda:
- Se verifica si `buffer` tiene contenido.
- De ser así, se crea la celda correspondiente y se agrega al notebook.

Este paso es fundamental para evitar la **pérdida de datos** al final del archivo.

In [7]:
if buffer: 
    contenido = "".join(buffer) 
    if current_type == "markdown": 
        nb.cells.append(nbformat.v4.new_markdown_cell(contenido)) 
    else: 
        nb.cells.append(nbformat.v4.new_code_cell(contenido)) 

## Guardado del notebook

Se guarda un archivo `.ipynb` con el contenido del notebook generado.

In [8]:
print(f"Celdas creadas: {len(nb.cells)}") 
 
 
with open(ipynb_path, "w", encoding="utf-8") as f: 
    nbformat.write(nb, f) 
 
print(f"Notebook creado: {ipynb_path}")

Celdas creadas: 7
Notebook creado: output/G4-notebook.ipynb
